In [1]:
from __future__ import division

import time
import math
import random

In [2]:
def randomPolicy(state):
    while not state.isTerminal():
        try:
            action = random.choice(state.getPossibleActions())
        except IndexError:
            raise Exception("Non-terminal state has no possible actions: " + str(state))
        state = state.takeAction(action)
    return state.getReward()

In [3]:
class treeNode():
    def __init__(self, state, parent):
        self.state = state
        self.isTerminal = state.isTerminal()
        self.isFullyExpanded = self.isTerminal
        self.parent = parent
        self.numVisits = 0
        self.totalReward = 0
        self.children = {}

In [5]:
class mcts():
    def __init__(self, timeLimit=None, iterationLimit=None, explorationConstant=1 / math.sqrt(2),
                 rolloutPolicy=randomPolicy):
        if timeLimit != None:
            if iterationLimit != None:
                raise ValueError("Cannot have both a time limit and an iteration limit")
            # time taken for each MCTS search in milliseconds
            self.timeLimit = timeLimit
            self.limitType = 'time'
        else:
            if iterationLimit == None:
                raise ValueError("Must have either a time limit or an iteration limit")
            # number of iterations of the search
            if iterationLimit < 1:
                raise ValueError("Iteration limit must be greater than one")
            self.searchLimit = iterationLimit
            self.limitType = 'iterations'
        self.explorationConstant = explorationConstant
        self.rollout = rolloutPolicy

    def search(self, initialState):
        self.root = treeNode(initialState, None)

        if self.limitType == 'time':
            timeLimit = time.time() + self.timeLimit / 1000
            while time.time() < timeLimit:
                self.executeRound()
        else:
            for i in range(self.searchLimit):
                self.executeRound()

        bestChild = self.getBestChild(self.root, 0)
        return self.getAction(self.root, bestChild)

    def executeRound(self):
        node = self.selectNode(self.root)
        reward = self.rollout(node.state)
        self.backpropogate(node, reward)

    def selectNode(self, node):
        while not node.isTerminal:
            if node.isFullyExpanded:
                node = self.getBestChild(node, self.explorationConstant)
            else:
                return self.expand(node)
        return node

    def expand(self, node):
        actions = node.state.getPossibleActions()
        for action in actions:
            if action not in node.children:
                newNode = treeNode(node.state.takeAction(action), node)
                node.children[action] = newNode
                if len(actions) == len(node.children):
                    node.isFullyExpanded = True
                return newNode

        raise Exception("Should never reach here")

    def backpropogate(self, node, reward):
        while node is not None:
            node.numVisits += 1
            node.totalReward += reward
            node = node.parent

    def getBestChild(self, node, explorationValue):
        bestValue = float("-inf")
        bestNodes = []
        for child in node.children.values():
            nodeValue = child.totalReward / (child.numVisits * node.state.loss()) + explorationValue * math.sqrt(
                2 * math.log(node.numVisits) / child.numVisits)
            if nodeValue > bestValue:
                bestValue = nodeValue
                bestNodes = [child]
            elif nodeValue == bestValue:
                bestNodes.append(child)
        return random.choice(bestNodes)

    def getAction(self, root, bestChild):
        for action, node in root.children.items():
            if node is bestChild:
                return action


# 路径规划

In [8]:
from __future__ import division
import numpy as np
from sys import exit
from copy import deepcopy
from mcts import mcts
from Dataset_G1 import create_data_model
from Resource_schedule import ResourceSchdule
import numpy as np
from functools import reduce
#from naughtsandcrosses import State

import copy
import itertools
import operator 

ModuleNotFoundError: No module named 'Dataset_G1'

In [ ]:
class RouterSchedule():
    def __init__(self, data):
        # coordinate of turbines in wind farms
        self.WF_coordinate = data['wind_farm']['coordinate']
        # num of wind farms
        self.B_coordinate = data['base']['coordinate']
        # fuel cost (/hour) of vessels
        self.V_cost = data['vessel']['cost']
        # transfer time for technicians and equipment from vessel to turbine
        self.V_transTime = data['vessel']['trans_time']
        # num. of needed technician of each type
        self.WF_tech = data['wind_farm']['technician']
        # speed (km/hour) of vessels
        self.V_speed = data['vessel']['speed']

        self.schdule_base = 1
        self.schdule_vessel = 1  # 需要调度的船的数目
        self.schdule_technician = 3  # 需要的维修人员的数目
        self.num_WF = len(self.WF_coordinate)
        self.state = self.getCurrentState()
        self.router_schdule_result = []
        self.cost_b_WF_v = []
        self.num_maintain = 2
        self.count = 0
        self.v_cost = 2



    def getCurrentState(self):
        state = [[[[0 for i in range (self.schdule_technician)]for j in range(self.schdule_vessel)] for k in range(self.schdule_base)]for m in range(self.num_WF)]
        #for i in range(len(state)):
        #   del state[i][self.schdule_vessel[i]:]#vessel
        #   for j in range(len(state[i])):
        #       del state[i][j][self.schdule_technician[i]:]

        return state

    def getPossibleActions(self):
        state = deepcopy(self.state)
        possibleAction = []
        for i in range(len(state)):  # t
            for j in range(len(state[i])):  # v
                for m in range(len(state[i][j])):  # b
                    for n in range(len(state[i][j][m])):  # kind #n = 0 tech, m=1 sp
                    #for k in range(len(state[i][j][m][n])):
                        if state[i][j][m][n] == 0:
                            possibleAction.append(Action(x=i, y=j, z=m, o=n))

        return possibleAction

    def takeAction(self, action):
        newState = deepcopy(self)
        newState.state[action.x][action.y][action.z][action.o] = 1
        #newState.currentPlayer = self.currentPlayer * -1
        return newState

    # to calculate distance including a base
    def createDistanceMatrix(self, coordinate, base):
        Num = len(coordinate) + 1
        d = np.zeros((Num, Num))
        for i in range(Num - 1):
            p1 = np.array([coordinate[i][0], coordinate[i][1]])
            p3 = np.array([base[0], base[1]])
            d[0, i + 1] = d[i + 1, 0] = np.linalg.norm(p1 - p3)
            for j in range(i + 1, Num - 1):
                p2 = np.array([coordinate[j][0], coordinate[j][1]])
                d[i + 1, j + 1] = d[j + 1, i + 1] = np.linalg.norm(p1 - p2)
        d = np.round(d, 2)
        # print(d)
        return d  # type: np.array

    def cost_v(self):
        for base in range(self.schdule_base):
            for farm in range(self.num_WF):
                for vessel in range(self.schdule_vessel):
                    # feasibility condition
                    # if self.V_available[vessel][t] == 0:
                    #     continue
                    # calculate the distance
                    dist = self.createDistanceMatrix(self.WF_coordinate[farm], self.B_coordinate[base])
                    # travel time between turbines
                    travelTime = dist / self.V_speed[vessel]
                    cost_b_WF_v = travelTime * self.v_cost
        return cost_b_WF_v[0]
    def loss(self):
        L = (self.cost_v())[1]
        return L

    def isTerminal(self):  # final terminal
        self.count += 1
        if self.count > self.num_maintain:
            return True
        return False

    def getReward(self):

        return 0.5

In [ ]:
class Action():
    def __init__(self, x, y, z, o):
        self.x = x
        self.y = y
        self.z = z
        self.o = o

    def __str__(self):
        return str((self.x, self.y, self.z, self.o))

    def __repr__(self):
        return str(self)

    def __hash__(self):
        return hash((self.x, self.y, self.z, self.o))

# 资源调度

In [6]:
from __future__ import division
import numpy as np
from sys import exit
from copy import deepcopy
from mcts import mcts
from dataset import create_data_model
import numpy as np
from functools import reduce

import copy
import itertools
import operator 

ModuleNotFoundError: No module named 'mcts'

In [ ]:
class ResourceSchdule():
    def __init__(self, data):
        # num of wind farms
        self.B_vessel = data['base']['vessel']
        # num. of available technician at bases
        self.B_tech = data['base']['technician']
        # num of wind farms
        self.B_coordinate = data['base']['coordinate']
        # coordinate of turbines in wind farms
        self.WF_coordinate = data['wind_farm']['coordinate']

        self.B_V = self.getBaseVessel()
        self.B_T = self.getBaseTechnician()
        # num of bases
        self.num_B = len(self.B_V)  # len(self.lambda_bf)#len(self.B_tech)
        self.num_V = self.B_V # len(self.time_window)
        self.num_T = self.B_T
        self.count = 0
        self.V_num = 2  # 需要调度的船的数目
        self.T_num = 3  #需要的维修人员的数目
        self.schdule_base = 0
        self.schdule_vessel = 0
        self.schdule_technician = 0
        self.state = self.getCurrentState()
        self.resource_schdule_result = []

    def getBaseVessel(self):
        Base_V_num = np.zeros(len(self.B_vessel), dtype=np.int)
        for i in range(len(self.B_vessel)):
            Base_V_num[i] = sum(self.B_vessel[i][:])

        return Base_V_num

    def getBaseTechnician(self):
        Base_T_num = np.zeros(len(self.B_tech), dtype=np.int)
        for i in range(len(self.B_tech)):
            Base_T_num[i] = sum(self.B_tech[i][:])
        return Base_T_num

    def getAvailableVessels(self):
        index = []
        nums = []
        for i in range(len(self.B_vessel)):
            num = 0
            for j in range(len(self.B_vessel[i])):
                if self.B_vessel[i][j] == 1:
                    index.append((i,j))
                    num += 1
            nums.append(num)
        print(index)
        return index, nums

    def getCurrentState(self):
        state = [[[0 for i in range (max(self.num_T))]for j in range(max(self.num_V))] for k in range(self.num_B)]
        for i in range(len(state)):
           del state[i][self.num_V[i]:]#vessel
           for j in range(len(state[i])):
               del state[i][j][self.num_T[i]:]

        return state

    def getPossibleActions(self):
        state = deepcopy(self.state)

        unpossibleActions = []
        for i in range(len(state)):  # b
            for j in range(len(state[i])):  # v
                for m in range(len(state[i][j])):  # m = 0 tech, m=1 sp
                    #for n in range(len(state[i][j][m])):
                    if state[i][j][m] == 1:
                        for k in range(len(state[i])):
                            if k != j:
                                unpossibleActions.append(Action(x=i, y=k, z=m))
        if unpossibleActions:
            for index in unpossibleActions:
                #print(index)
                state[index.x][index.y][index.z] = 'un'

        possibleAction = []
        for i in range(len(state)):
            for j in range(len(state[i])):
                for k in range(len(state[i][j])):
                    if state[i][j][k] == 0:
                        possibleAction.append(Action(x=i, y=j, z = k))

        return possibleAction

    def takeAction(self, action):
        newState = deepcopy(self)
        newState.state[action.x][action.y][action.z] = 1
        #newState.currentPlayer = self.currentPlayer * -1
        return newState

    def cost_v(self):
        cost_b_WF_v = [[1,1],[1,0]]
        return cost_b_WF_v[0][1]
    def loss(self):
        L = self.cost_v()

        return L


    def isTerminal(self):
        self.count += 1
        if self.count > self.T_num:
            return True
        return False

    def getReward(self):

        return 0.5

    def schdule(self):
        schdule_base = self.resource_schdule_result[0].x
        schdule_vessel = self.resource_schdule_result[0].y
        schdule_technician = self.resource_schdule_result[0].z
        schdule_base = 0
        schdule_vessel = 0
        schdule_technician = 0
        for i in range(len(self.resource_schdule_result)):
            if self.resource_schdule_result[i].x == schdule_base:
                schdule_base += 1
            if self.resource_schdule_result[i].y == schdule_vessel:
                schdule_vessel += 1
            if self.resource_schdule_result[i].z == schdule_technician:
                schdule_technician += 1
        return schdule_base, schdule_vessel, schdule_technician
        #print(schdule_base, schdule_vessel, schdule_technician)

In [ ]:
'''定义动作集合，即怎样表示了一个动作'''
class Action():
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z



    def __str__(self):
        return str((self.x, self.y, self.z))

    def __repr__(self):
        return str(self)

    def __hash__(self):
        return hash((self.x, self.y, self.z))




'''
def resource_schdule():

    data = create_data_model()
    S = State(data)
    print('initialState:', S.state)
    # print(len(initialState.state[0][0][0]))
    a_set = S.getPossibleActions()
    #v = S.getAvailableVessels()
    print('动作集:', a_set)  #
    count = 0
    while count < S.T_num:
        # 机器下棋
        action = mcts.search(initialState=S)
        S.resource_schdule_result.append(action)
        print('调度动作', action)
        S = S.takeAction(action)
        print('采取动作之后的形成的新的状态：', S.state)
        a_set = S.getPossibleActions()
        print('新的动作集合:', a_set)
        count += 1 #需要配置的技术人员数目
    print('资源分配阶段的最终规划结果：', S.resource_schdule_result)
    S.schdule()
    return S.schdule()


if __name__ == "__main__":
    mcts = mcts(timeLimit=1000)
    resource_schdule()
'''